# Cleaning

## Setup

In [4]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

# import data
full_data = pd.read_csv("Rail_Equipment_Accident_Incident_Data.csv")
full_data.head()

/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (27,43,46,47,48,50,59,61,62,63,64,65,66,67,68,69,71,130,131,136) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Reporting Railroad Code,Reporting Railroad Name,Report Year,Accident Number,PDF Link,Accident Year,Accident Month,Other Railroad Code,Other Railroad Name,Other Accident Number,...,Other Railroad SMT Grouping,Other Parent Railroad Company Code,Other Parent Railroad Company Name,Other Railroad Holding Company,Maintenance Railroad Company Grouping,Maintenance Railroad Class,Maintenance Railroad SMT Grouping,Maintenance Parent Railroad Company Code,Maintenance Parent Railroad Company Name,Maintenance Railroad Holding Company
0,NS,Norfolk Southern Railway Company,2016.0,120068,https://safetydata.fra.dot.gov/Officeofsafety/...,16.0,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Class 1,SMT-3 - Norfolk Southern,NS,Norfolk Southern Railway Company,Norfolk Southern Railway Company
1,NS,Norfolk Southern Railway Company,2016.0,120068,https://safetydata.fra.dot.gov/Officeofsafety/...,16.0,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Class 1,SMT-3 - Norfolk Southern,NS,Norfolk Southern Railway Company,Norfolk Southern Railway Company
2,CR,Conrail,1981.0,0420001,https://safetydata.fra.dot.gov/Officeofsafety/...,81.0,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Class 3,NaN,CRSH,Consolidated Rail Corporation,Not Assigned
3,NS,Norfolk Southern Railway Company,2016.0,120161,https://safetydata.fra.dot.gov/Officeofsafety/...,16.0,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Class 1,SMT-3 - Norfolk Southern,NS,Norfolk Southern Railway Company,Norfolk Southern Railway Company
4,NS,Norfolk Southern Railway Company,2016.0,120161,https://safetydata.fra.dot.gov/Officeofsafety/...,16.0,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Class 1,SMT-3 - Norfolk Southern,NS,Norfolk Southern Railway Company,Norfolk Southern Railway Company


## Dropping Features

In [5]:
features_to_drop = ["Reporting Railroad Code",
                 "PDF Link",
                 "Other Railroad Code",
                 "Maintenance Railroad Code",
                 "Accident Type Code",
                 "State Code",
                 "County Code",
                 "Visibility Code",
                 "Weather Condition Code",
                 "Track Type Code",
                 "Train Direction Code",
                 "Equipment Type Code",
                 "Signalization Code",
                 "Method of Operation Code",
                 "Remote Control Locomotive Code",
                 "Primary Accident Cause Code",
                 "Contributing Accident Cause Code",
                 "Accident Cause Code",
                 "Class Code",
                 "Reporting Parent Railroad Company Code",
                 "Other Parent Railroad Company Code",
                 "Maintenance Parent Railroad Company Code"
]

data = full_data.drop(features_to_drop,axis=1)
data.head()

,Reporting Railroad Name,Report Year,Accident Number,Accident Year,Accident Month,Other Railroad Name,Other Accident Number,Other Accident Year,Other Accident Month,Maintenance Railroad Name,...,Other Railroad Company Grouping,Other Railroad Class,Other Railroad SMT Grouping,Other Parent Railroad Company Name,Other Railroad Holding Company,Maintenance Railroad Company Grouping,Maintenance Railroad Class,Maintenance Railroad SMT Grouping,Maintenance Parent Railroad Company Name,Maintenance Railroad Holding Company
0,Norfolk Southern Railway Company,2016.0,120068,16.0,4.0,NaN,NaN,NaN,NaN,Norfolk Southern Railway Company,...,NaN,NaN,NaN,NaN,NaN,NaN,Class 1,SMT-3 - Norfolk Southern,Norfolk Southern Railway Company,Norfolk Southern Railway Company
1,Norfolk Southern Railway Company,2016.0,120068,16.0,4.0,NaN,NaN,NaN,NaN,Norfolk Southern Railway Company,...,NaN,NaN,NaN,NaN,NaN,NaN,Class 1,SMT-3 - Norfolk Southern,Norfolk Southern Railway Company,Norfolk Southern Railway Company
2,Conrail,1981.0,0420001,81.0,4.0,NaN,NaN,NaN,NaN,Conrail,...,NaN,NaN,NaN,NaN,NaN,NaN,Class 3,NaN,Consolidated Rail Corporation,Not Assigned
3,Norfolk Southern Railway Company,2016.0,120161,16.0,4.0,NaN,NaN,NaN,NaN,Norfolk Southern Railway Company,...,NaN,NaN,NaN,NaN,NaN,NaN,Class 1,SMT-3 - Norfolk Southern,Norfolk Southern Railway Company,Norfolk Southern Railway Company
4,Norfolk Southern Railway Company,2016.0,120161,16.0,4.0,NaN,NaN,NaN,NaN,Norfolk Southern Railway Company,...,NaN,NaN,NaN,NaN,NaN,NaN,Class 1,SMT-3 - Norfolk Southern,Norfolk Southern Railway Company,Norfolk Southern Railway Company


In [4]:
# data["Accident Cause"].head(5)
s = "JOB SHOVING 8 CARS INTO 121 TRACK DERAILED TWO CARS AND TWO ENGINES JUST EAST OF 121 SWITCH"
# find the column that contains s
data.columns[data.isin([s]).any()]

Index([], dtype='object')

In [5]:
# Dropping additional columns here that we won't initially need
data = data.drop(["Accident Cause"],axis=1)

## Feature Engineering/Cleaning

In [6]:
# supposedly these are columns with multiple data types, need further examination
df_test = data.iloc[:,[22,36,37,46,47,48,49,50,51,52,53,54,56,112,113,118]]
df_test.head()

,Subdivision,Track Class,Track Density,Signalization,Method of Operation,Adjunct Code 1,Adjunct Code Name 1,Adjunct Code 2,Adjunct Code Name 2,Adjunct Code 3,Adjunct Code Name 3,Remote Control Locomotive,First Car Number,Special Study 2,Latitude,Class
0,SYSTEM,1,NaN,Not Signaled,Other Than Main Track,NaN,NaN,NaN,NaN,NaN,NaN,Not a remotely controlled operation,000203,000-000-000,41.884035,No
1,SYSTEM,1,NaN,Not Signaled,Other Than Main Track,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,000-000-000,41.884035,No
2,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,900064,NaN,NaN,1L
3,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not a remotely controlled operation,002000,NaN,0.000000,1L
4,CHICAGO,1,NaN,Not Signaled,Other Than Main Track,NaN,NaN,NaN,NaN,NaN,NaN,Not a remotely controlled operation,668680,000-000-000,41.054254,Cl


In [11]:
# Handling NaNs first 
# remove columns with > 50,000 NaNs
# set remaining NaNs to mean for numerical data, encode "NA" for categorical
na_counts = data.isna().sum()
cols_to_drop = na_counts[na_counts > 50000].index.tolist()
data.drop(cols_to_drop, axis=1, inplace=True)

#split data into numerical and categorical data to process
# quanititative features
num_data = data.select_dtypes(include=['float64', 'int64'])
num_data.fillna(num_data.mean(), inplace=True)
print(num_data.isna().sum().sort_values(ascending=False))

# qualititative features
cat_data = data.select_dtypes(include=['object'])
cat_data.fillna('NA', inplace=True)

print(cat_data.isna().sum().sort_values(ascending=False))


/opt/anaconda3/lib/python3.9/site-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


Report Year                               0
Empty Passenger Cars                      0
Derailed Loaded Freight Cars              0
Derailed Loaded Passenger Cars            0
Derailed Empty Freight Cars               0
Derailed Empty Passenger Cars             0
Derailed Cabooses                         0
Equipment Damage Cost                     0
Track Damage Cost                         0
Total Damage Cost                         0
Engineers On Duty                         0
Firemen On Duty                           0
Conductors On Duty                        0
Brakemen On Duty                          0
Hours Engineers On Duty                   0
Railroad Employees Killed                 0
Railroad Employees Injured                0
Passengers Killed                         0
Passengers Injured                        0
Others Killed                             0
Others Injured                            0
Persons Killed For Reporting Railroad     0
Persons Injured For Reporting Ra

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:5176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


Reporting Railroad Name                     0
Accident Number                             0
First Car Initials                          0
First Car Number                            0
First Car Loaded                            0
Passengers Transported                      0
Primary Accident Cause                      0
Accident Cause                              0
Narrative                                   0
Joint Track Type                            0
Joint Track Class                           0
Class                                       0
Incident Key                                0
Report Key                                  0
Reporting Railroad Class                    0
Reporting Railroad SMT Grouping             0
Reporting Parent Railroad Company Name      0
Reporting Railroad Holding Company          0
Maintenance Railroad Class                  0
Maintenance Railroad SMT Grouping           0
Maintenance Parent Railroad Company Name    0
Recorded Estimated Speed          

# Cleaning for Neural Network

In [7]:
data_nn = data

# Accident Year, Accident Month, Maintenence Railroad name, State Name, Visibility, Train Speed, Total Damage Cost
data_nn = data_nn[["Accident Year", "Accident Month", "Maintenance Railroad Name", "State Name", "Visibility", "Train Speed", "Total Damage Cost"]]


# Save the data to a csv file
data_nn.to_csv("data_nn.csv", index=False)


## Regularization

## Saving

In [7]:
# save data to file
data.to_csv("cleaned_data.csv",index=False) # index=False to not save the index column

In [8]:
# saves all column names to separate file in order to find different variables
with open("column_names.txt", "w") as f:
    for s in data.columns:
        f.write(s + "\n")